In [1]:
import tkinter as tk
import re
import ipynb_importer
from publish_subscribe import Reader
import pickle
import pandas as pd
import numpy as np

class Poem():
    def __init__(self,_id, _t, _a = "", _c = ()):
        self.id = _id
        self.title = _t
        self.author = _a
        self.content = _c
        
    def __str__(self):
        msg = f"{self.author}《{self.title}》\n\n"
        for i in self.content:
            msg += i+"\n\n"
        return msg

importing Jupyter notebook from publish_subscribe.ipynb


In [2]:
class MainPanel:
    def __init__(self, reader, publisher):
        self.reader = reader
        self.publisher = publisher
        # Getting back the objects:
        with open('objs.pkl','rb') as f:  # Python 3: open(..., 'rb')
            self.rel,self.word2freq,self.word_rel,self.doc_rel,self.poem2idx,self.word2idx,self.idx2poem,self.idx2word,self.terms_df,self.publisher.ipoems,self.publisher.apoems = pickle.load(f)
        
    """************************************
     显示主界面
    ************************************"""
    def start(self):
        self.root = tk.Tk()
        self.root.geometry('600x600')
        self.root.title("古诗订阅")
        self.root.iconbitmap('cat.ico')
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12)).pack(pady=20)
        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12), command=self.add_subscription).pack(pady=40)
        self.root.mainloop()
        
    """************************************
     用户点击"新增订阅"按钮，弹出该界面
    ************************************"""
    def add_subscription(self):
        self.add_panel = tk.Tk()
        self.add_panel.geometry('600x400')
        self.add_panel.iconbitmap('cat.ico')
        self.add_panel.title("新增订阅")
        
        #已订阅作者
        present_author_label = tk.Label(self.add_panel, text=f"已订阅作者：{self.reader.authors}", font=(None,10)).pack(pady=20)
        
        #新增订阅
        label = tk.Label(self.add_panel, text="新增订阅:", font=(None, 10)).pack(pady=20)
        self.new_author_entry = tk.Entry(self.add_panel)
        self.new_author_entry.pack()
        
        #关键词
        keyword = tk.Label(self.add_panel, text="关键词:", font=(None, 10)).pack(pady=20)
        self.new_keyword_entry = tk.Entry(self.add_panel)
        self.new_keyword_entry.pack()
        
        confirm_button = tk.Button(self.add_panel, text="确认", font=(None, 10), command=self.check_new_author).pack(pady=20)
        self.hint_label = tk.Label(self.add_panel, text="", font=(None, 10))
        self.hint_label.pack()
    
    """************************************
     用户点击"确认"按钮后，检查输入是否合法
    ************************************"""
    def check_new_author(self):
        new_author = self.new_author_entry.get()
        new_keywords = self.new_keyword_entry.get().split()
        for i in new_keywords:
            if i not in self.word2freq.keys():
                self.hint_label.config(text=f"目前不支持订阅关键词{i}")
                return
        if new_author != "" and new_author not in self.publisher.apoems.keys():
            self.hint_label.config(text=f"目前不支持订阅{new_author}")
        elif new_author in self.reader.authors:
            self.hint_label.config(text="该作者已订阅，请重新输入")
        else:
            tmp_tk = tk.Tk()
            tmp_tk.geometry("300x200")
            tmp_tk.title("确认面板")
            tk.Label(tmp_tk, text="订阅成功", font=(None, 15)).pack(pady=20)
            tk.Button(tmp_tk, text="确认", font=(None, 12), command=tmp_tk.destroy).pack(pady=20)
            self.reader.subscribeToPublisher(new_author, self.publisher, self.new_keyword_entry.get())
            self.add_panel.destroy()
            self.update_main_panel()
    
    """************************************
     用户新增订阅后，更新主界面
    ************************************"""
    def update_main_panel(self):
        for widget in self.root.winfo_children():
            widget.destroy()
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12)).pack(pady=20)
        for sub in self.reader.subs:
            msg = f"【作者】{sub[0]} " if sub[0] != "" else ""
            if sub[0] != "" and sub[1] != "":
                msg += "，"
            msg += f"【关键词】{sub[1]}" if sub[1] != "" else ""
            button = tk.Button(self.root, text=msg,width=60, font=(None,12))
            button.bind("<Button-1>", lambda event, x=sub[0],y=sub[1]:self.show_titles(x,y))
            button.pack(pady=10)
        self.filter_label = tk.Label(self.root, text="过滤精度", font=(None,10)).pack(pady=10)
        self.filter_entry = tk.Entry(self.root)
        self.filter_entry.insert(tk.END, 0.5)
        self.filter_entry.pack()
        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12), command=self.add_subscription).pack(pady=20)
        self.root.update()
        
    """************************************
     近义词挖掘
    ************************************"""
    def get_similar(self,keyword):
        res = {}
        f = float(self.filter_entry.get())
        num = self.word2idx[keyword]
        lst = self.rel[num].toarray().tolist()[0]
        lst = [(x,y) for x,y in zip(range(len(lst)),lst)] #word_id, cov(num,word_id)
        lst.sort(reverse=True,key=lambda x:x[1])
        x2 = 10
        ratio = 1
        for i in lst: #选最接近的字
            if i[1] < 0.68739376*f: #选出前0.05%匹配的字（当f=100%）
                break
            ratio *= 0.78
            res[self.idx2word[i[0]]]=[]
            tmp = self.terms_df.loc[self.idx2word[i[0]]].sort_values(ascending=False)
            for idx,j in zip(range(x2),tmp.head(x2).index): # (tf-idf,Poem_id)
                if(tmp[idx] == 0):
                    break
                res[self.idx2word[i[0]]].append((j[1],(i[1]+tmp[idx])*ratio))
        return res
    
    """************************************
     用户点击“订阅项目”按钮后，弹出订阅内容
    ************************************"""       
    def show_titles(self, author, keywords):
        self.add_panel = tk.Tk()
        self.add_panel.geometry('600x400')
        self.add_panel.iconbitmap('cat.ico')
        k = " ".join(keywords)
        self.add_panel.title(f"{author} {k}")
        
        #关键词分组
        keywords = keywords.split()
        poems_by_keywords = {}
        for i in keywords:
            if self.word2freq[i] > 10:
                poems_by_keywords.update(self.get_similar(i))
    
        klst = [] #(poem_id,高亮词）
        for i in poems_by_keywords.keys():
            for j in poems_by_keywords[i]:
                klst.append(j+(i,))
        klst.sort(reverse=True,key=lambda x:x[1]) 
        klst = [(x[0],x[2]) for x in klst]
        
        #作者分组
        alst = [] #(poem_id,高亮词）
        if(author != ""):
            for i in self.reader.poems[author]: #该作者的所有诗歌
                alst.append((i.id,author))

        #目前显示的诗歌集合
        self.poems = {} #{poem_id:高亮词}
        self.idx2title = {}
        self.title2idx = {}
        
        #取最终要选的诗歌集合
        res = []
        if not alst:
            res = klst
        elif not klst:
            res = alst
        else:
            constraint = [x[0] for x in alst]
            for i in klst:
                if(i[0] in constraint):
                    res.append(i)
        
        #消除重複:
        for i in res:
            if i[0] in self.poems:
                self.poems[i[0]].append(i[1])
            else:
                self.poems[i[0]] = [i[1]]
                
        #只要置信度最高的前50个
        res = {}        
        if len(self.poems.keys()) > 50:
            for i,j in zip(range(50),self.poems.keys()):
                res[j] = self.poems[j]
            self.poems = res
        
        #滑动栏
        ScrollBar = tk.Scrollbar(self.add_panel)
        ScrollBar.pack(side=tk.RIGHT, fill = tk.Y)
        
        #搜索栏
        self.new_title_entry = tk.Entry(self.add_panel)
        self.new_title_entry.pack()
        self.hint_label = tk.Label(self.add_panel, text="", font=(None, 10))
        self.hint_label.pack()
        
        #确认搜索
        confirm_button = tk.Button(self.add_panel, text="搜索", font=(None, 10), command=self.check_new_title).pack(pady=20)
        
        #诗歌内容
        ListBox = tk.Listbox(self.add_panel, width=65, height=20, font=(None,12), yscrollcommand=ScrollBar.set)
        for idx,i in zip(range(len(self.poems.keys())),self.poems.keys()): #标题
            title = self.publisher.ipoems[i].title
            ListBox.insert(tk.END, title)
            self.idx2poem[idx] = (i,self.poems[i])
            self.idx2title[idx]=title 
            self.title2idx[title]=idx 
            self.max_length = idx
        ListBox.bind('<<ListboxSelect>>',lambda event,x=ListBox:self.handler(x))
        ListBox.pack(side=tk.LEFT, fill=tk.BOTH)
        self.add_panel.mainloop()
    
    def handler(self, x):
        title = x.get(x.curselection())
        self.show_detail(title)
    
    """************************************
     用户点击"确认"按钮后，检查输入是否合法
    ************************************"""
    def check_new_title(self):
        new_title = self.new_title_entry.get()
        if new_title not in self.title2idx.keys():
            self.hint_label.config(text=f"查无此标题")
        else:
            self.show_detail(new_title)
    
    """************************************
     用户点击"诗歌标题"按钮后，弹出完整内容
    ************************************"""
    def show_detail(self, title):
        self.idx=self.title2idx[title]
        poem = self.publisher.ipoems[self.idx2poem[self.idx][0]]
        hl = self.idx2poem[self.idx][1]
        author = poem.author
        content = f"{poem.id} {poem}"
        self.add_detail = tk.Tk()
        self.add_detail.geometry('700x600')
        self.add_detail.iconbitmap('cat.ico')
        self.add_detail.title(title)
        tk.Label(self.add_detail,text=content,font=(None,15)).pack(pady=5)
        tk.Label(self.add_detail,text=hl,font=(None,15), bg='yellow').pack(pady=5)
        previous_button = tk.Button(self.add_detail, text="上一页", font=(None, 15), command=self.prev_page).pack(pady=20)
        next_button = tk.Button(self.add_detail, text="下一页", font=(None, 15), command=self.next_page).pack(pady=20)
        self.hint_label = tk.Label(self.add_detail, text="", font=(None, 15))
        self.hint_label.pack()
            
    """************************************
     用户点击"上/下一页"按钮后，更新内容页面
    ************************************"""
    def next_page(self):
        idx = self.idx
        if idx == self.max_length:
            self.hint_label.config(text=f"已是最后一页!")
        else:
            self.update_detail(idx+1)
    
    def prev_page(self):
        idx=self.idx
        if idx == 0:
            self.hint_label.config(text=f"已是第一页！")
        else:
            self.update_detail(idx-1)
        
    def update_detail(self, idx):
        title = self.idx2title[idx]
        self.idx=idx
        poem = self.publisher.ipoems[self.idx2poem[self.idx][0]]
        hl = self.idx2poem[self.idx][1]
        author = poem.author
        content = f"{poem.id} {poem}"
        for widget in self.add_detail.winfo_children():
            widget.destroy()
        self.add_detail.geometry('700x600')
        self.add_detail.iconbitmap('cat.ico')
        self.add_detail.title(title)
        tk.Label(self.add_detail,text=content,font=(None,15)).pack(pady=5)
        tk.Label(self.add_detail,text=hl,font=(None,15), bg='yellow').pack(pady=5)
        previous_button = tk.Button(self.add_detail, text="上一页", font=(None, 15), command=self.prev_page).pack(pady=20)
        next_button = tk.Button(self.add_detail, text="下一页", font=(None, 15), command=self.next_page).pack(pady=20)
        self.hint_label = tk.Label(self.add_detail, text="", font=(None, 15))
        self.hint_label.pack()
        